In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interactive
import lightkurve as lk
from bokeh.models import Select, Row, ColumnDataSource, Slider
from bokeh.plotting import figure, curdoc
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row, widgetbox
from bokeh.themes import built_in_themes
from bokeh.models import Button, CustomJS
from bokeh.models import Span
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.models import RangeSlider
from streamlit import caching
from scipy.signal import savgol_filter
from os.path import dirname, join, realpath
#import os
#import pathlib

#%matplotlib inline

In [2]:
TIC_list = ['172900988', '454140642', ]
Source_list = ['MAST QLP', 'Eleanor (local)']

In [3]:
def get_data_archive(TIC):
    lcs_QLP = lk.search_lightcurve('TIC ' + str(TIC), author = 'QLP').download_all(quality_bitmask="none");

    time_QLP, flux_QLP, flux_QLP_err, flags_QLP, COM_x, COM_y, sap_bkg, sector = [], [], [], [], [], [], [], []
    for ii in range(len(lcs_QLP.sector)):
        time_QLP = np.hstack((time_QLP,lcs_QLP[ii].time.value))
        flux_QLP = np.hstack((flux_QLP,lcs_QLP[ii].flux.value))
        flux_QLP_err = np.hstack((flux_QLP_err,lcs_QLP[ii].flux_err.value))
        flags_QLP = np.hstack((flags_QLP,lcs_QLP[ii].quality.value))
        COM_x = np.hstack((COM_x, lcs_QLP[ii].sap_x.value/np.nanmedian(lcs_QLP[ii].sap_x.value)))
        COM_y = np.hstack((COM_y, lcs_QLP[ii].sap_y.value/np.nanmedian(lcs_QLP[ii].sap_y.value)))
        sap_bkg = np.hstack((sap_bkg, lcs_QLP[ii].sap_bkg.value/np.nanmean(lcs_QLP[ii].sap_bkg.value)))
        sector = np.hstack((sector, lcs_QLP[ii].sector + np.zeros(len(lcs_QLP[ii].time))))
         
    flags_QLP = flags_QLP#.astype(bool);

    df = pd.DataFrame(data={
        "time"     : time_QLP,
        "corr flux"     : flux_QLP,
        "flags"     :  flags_QLP,
        "PSF x-com"     : COM_x,
        "PSF y-com"     : COM_y,
        "PSF x-width"     :  np.ones((len(COM_y))),
        "PSF y-width"     :  np.ones((len(COM_y))),
        "PSF rot"     :  np.ones((len(COM_y))),
        "PSF bkg"     :  1.+sap_bkg,
        "Sector"     :  sector.astype(int),
        })
    return df

In [4]:
def get_data_local(TIC):

#    fname_LC = join(os.path.dirname(os.path.realpath('__file__')), 'data/TIC_' + str(TIC) + '_eleanor_LC.csv')
    fname_LC = join(dirname(realpath('__file__')), 'data/TIC_' + str(TIC) + '_eleanor_LC.csv')    
    data_LC = pd.read_csv(fname_LC, header = 0, delimiter=r",", na_filter= True, engine = 'python').values
    
    fname_LC_diag = join(dirname(realpath('__file__')), 'data/TIC_' + str(TIC) + '_eleanor_LC_diag.csv')
    data_LC_diag = pd.read_csv(fname_LC_diag, header=0, delimiter=r",", na_filter= True, engine = 'python').values

    df_LC = pd.DataFrame(data={
        "time"     : data_LC[:,0],
        "raw flux"     : data_LC[:,1],
        "corr flux"     :  data_LC[:,2],
        "flags"     :  np.zeros((len(data_LC[:,0]))),#.astype(bool),
        "pca flux"     :  data_LC[:,3],
        "psf flux"     :  data_LC[:,4],
        "PSF x-com"     : data_LC_diag[:,0],
        "PSF y-com"     : data_LC_diag[:,1],
        "PSF x-width"     :  data_LC_diag[:,2],
        "PSF y-width"     :  data_LC_diag[:,3],
        "PSF rot"     :  data_LC_diag[:,4],
        "PSF bkg"     :  data_LC_diag[:,5],
        "Sector"     :  np.zeros((len(data_LC[:,0]))),
        })

    return df_LC

In [5]:
output_notebook();

Loading BokehJS ...

In [6]:
#def plot_func(TIC, Source):
def plot_func(TIC, Source_):
#    fig = plt.figure(figsize=(20,5))
#    plt.rcParams.update({'font.size': 25})

    TIC = str(TIC)

    if Source_ == Source_list[0]:
        df_LC = get_data_archive(TIC).copy()
    elif Source_ == Source_list[1]:
        df_LC = get_data_local(TIC).copy()
    
#    datasource = ColumnDataSource({'x': x, 'y0': y0, 'y1': y1, 'y2': y2})
    
#    distribution = 'Discrete'
#    distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])
    
    time, flux, flags = df_LC['time'].values, df_LC['corr flux'].values, df_LC['flags'].values
    
    psf_xcom, psf_ycom, psf_xwidth, psf_ywidth, psf_rot, psf_bkg, sector = \
    df_LC['PSF x-com'].values, df_LC['PSF y-com'].values, \
    df_LC['PSF x-width'].values, df_LC['PSF y-width'].values, \
    df_LC['PSF rot'].values, df_LC['PSF bkg'].values, df_LC['Sector'].values
    
    if Source_ == 'MAST QLP':   
        spoc_flags = 4096. + 32 + np.asarray([0, 1, 2, 4, 8, 16, 32, 128, 1024, 2048])
        md = np.zeros(len(flags)).astype(bool)
        for ii in range(len(flags)):
            for jj in range(len(spoc_flags)):
                if flags[ii] == spoc_flags[jj]:
                    md[ii] = True
    else:
        md = np.zeros(len(flags)).astype(bool)
        
    time_MD = time[md]
    
    tools="pan,box_zoom,wheel_zoom,reset,freehand_draw,save"#,xbox_select,reset,freehand_draw, save'
    
    p = figure(width=900, height=250, title="TIC " + str(TIC) + ' LC', \
               x_range=(min(time),max(time)), \
#               y_range=(0.5,1.2), \
               y_axis_label='Normalized Flux', tools = tools, active_drag="pan",active_scroll='wheel_zoom')#,\
#               tools=tools, active_drag="xbox_select")
    source = ColumnDataSource(data=dict(x=time[~md], y=flux[~md]))
#    p.line(time[~md], flux[~md], legend_label="LC", line_width=2, color="white");
#    p.circle(time[~md], flux[~md], size = 2, color="white");
    p.line('x', 'y', source=source, legend_label="LC", line_width=2, color="white");
    p.circle('x', 'y', source=source, size = 2, color="white");
    
    x_range_slider = RangeSlider(start=time[~md][0], end=time[~md][0]+30, \
                                 value=(10.+1.*time[~md][0],0.+1.*time[~md][0]+20.), step=10)
        
#    x_range_slider = RangeSlider(start=time[~md][0], end=time[~md][0]+30, step=10, value=(p.x_range.start, p.x_range.end))
    
    x_range_slider.js_link("value", p.x_range, "start", attr_selector=0)
    x_range_slider.js_link("value", p.x_range, "end", attr_selector=1)
    
#    layout = column(x_range_slider, p)
#    show(layout)
    
    
    x_range_slider.tags = [dict(prior_val=x_range_slider.value, counter=0)]

    
    
    

#    slider = Slider(title="offset", value=0.0, start=-15.0, end=15.0, step=0.1)
#    callback = CustomJS(args=dict(source=source, val=slider),
#                    code="""
#        const data = source.data;
#        const freq = val.value;
#        const x = data['x'];
#        const y = data['y'];
#        for (var i = 0; i < x.length; i++) {
#            x[i] = freq + x[i];
#        }      
#        source.change.emit();
#    """)

#    slider.js_on_change('value', callback)
#    layout = column(slider, p)
#    show(layout)
    
    for kk in range(len(time_MD)):
        vline = Span(location = time_MD[kk], dimension='height', line_color='red', line_alpha =0.1,line_width=12)
        p.renderers.extend([vline])
    
    p.legend.location = "bottom_right"
    

    
                    
#    from bokeh.models import FreehandDrawTool
#    tool = FreehandDrawTool(renderers=[p])
    
    p1 = figure(width=900, height=250, title="TIC " + str(TIC) + ' COM', \
                y_range=(0.99,1.01), \
                y_axis_label='PSF COM', x_range=p.x_range,active_scroll='wheel_zoom')#, tools = tools)#,\
#                tools=tools, active_drag="xbox_select")
    p1.line(time[~md], psf_xcom[~md], legend_label="XCOM", line_width=2, color="red");
    p1.line(time[~md], psf_ycom[~md], legend_label="YCOM", line_width=2, color="lightgreen");
    for kk in range(len(time_MD)):
        vline = Span(location = time_MD[kk], dimension='height', line_color='red',line_alpha =0.1,line_width=12)
        p1.renderers.extend([vline])
    p1.legend.location = "bottom_right"
    
    p2 = figure(width=900, height=250, title="TIC " + str(TIC) + ' PSF Width', \
                y_range=(0.8,1.2), \
                y_axis_label='PSF Width', x_range=p.x_range,active_scroll='wheel_zoom')#, tools = tools)#,\
#               tools=tools, x_range=p.x_range)
    p2.line(time[~md], psf_xwidth[~md], legend_label="x-width", line_width=2, color="blue");
    p2.line(time[~md], psf_ywidth[~md], legend_label="y-width", line_width=2, color="brown");
    for kk in range(len(time_MD)):
        vline = Span(location = time_MD[kk], dimension='height', line_color='red', line_alpha =0.1,line_width=12)
        p2.renderers.extend([vline])
    p2.legend.location = "bottom_right"  
    
    p3 = figure(width=900, height=250, title="TIC " + str(TIC) + ' PSF Rot + Bkg', \
                y_range=(0.5,1.5), \
                x_axis_label='Time [BJD-2,457,000]', \
                y_axis_label='PSF rot + bkg', x_range=p.x_range,active_scroll='wheel_zoom')#, tools = tools)#,\
#                tools=tools, active_drag="xbox_select")
    p3.line(time[~md], psf_rot[~md], legend_label="Rotation", line_width=2, color="cyan");
    p3.line(time[~md], psf_bkg[~md], legend_label="Background", line_width=2, color="magenta");
    for kk in range(len(time_MD)):
        vline = Span(location = time_MD[kk], dimension='height', line_color='red', line_alpha =0.1,line_width=12)
        p3.renderers.extend([vline])
    p3.legend.location = "bottom_right"  
    
    p4 = figure(width=900, height=250, title="TIC " + str(TIC) + ' PSF Rot + Bkg')
    tpf = 1.
    
    
#    tpf_lk = lk.search_tesscut('TIC ' + str(int(TIC)), sector=21).download_all(cutout_size = (13,13))
    
#    p4 = figure(width=300, height=300)#, title="TIC " + str(TIC) + ' PSF Rot + Bkg', \
#                y_range=(0.5,1.5), \
#                x_axis_label='Time [BJD-2,457,000]', \
#                y_axis_label='PSF rot + bkg', x_range=p.x_range)
#    plt.imshow(np.nanmedian(tpf_lk[0], axis = 0))
    
    layout = column(x_range_slider, p, p1, p2, p3)
    show(layout)#column(x_range_slider, p, p1, p2, p3))#, p4))
    
    def amp_tags_cb():
        if x_range_slider.tags[0]['prior_val'] != x_range_slider.value:
            x_range_slider.tags = [dict(prior_val=x_range_slider.value, counter=x_range_slider.tags[0]['counter']+1)]

    curdoc().add_periodic_callback(amp_tags_cb, period_milliseconds=3000)
    curdoc().add_root(layout)
    
#    column(p,p1,p2,p3)
#    column(children=[widgets, plot], sizing_mode='stretch_both')
    
    del df_LC,p,p1,p2,p3,p4,time,flux,psf_xcom,psf_ycom,psf_xwidth,psf_ywidth,psf_rot,psf_bkg,flags,sector,tpf,\
    layout,x_range_slider
    caching.clear_cache()
#
curdoc().theme = 'dark_minimal'


#TIC = input('TIC (Default = 172900988): ')
#if len(TIC) == 0:
#    TIC = 172900988
#TIC = str(TIC)

interactive_plot = interactive(plot_func, TIC = TIC_list, Source_ = Source_list)#, a=(0,30,0,1)#, f=(0.1, 1))
output = interactive_plot.children[-1]
output.layout.height = '1200px'
interactive_plot

#del interactive_plot, output
#caching.clear_cache()

interactive(children=(Dropdown(description='TIC', options=('172900988', '454140642'), value='172900988'), Drop…

In [9]:
del interactive_plot, output
caching.clear_cache()

NameError: name 'interactive_plot' is not defined

In [ ]:
#tpf = lk.search_tesscut('TIC ' + str(int(454140642))).download_all(quality_bitmask=None, cutout_size=(13,13))

In [ ]:
#tpf[0].interact();

In [ ]:
#tpf[0][500:].interact_sky(magnitude_limit = 21);

In [ ]:
#tpf

In [ ]:
#tpf[0].flux
#cube = tpf[0].flux
#cube.shape
#plt.imshow(np.nanmedian(cube, axis = 0))